In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv

def create_database():
    conn = sqlite3.connect('gdp_by_country.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS gdp_by_country_data (id INTEGER PRIMARY KEY, country TEXT UNIQUE, gdp REAL, population INTEGER, gdp_per_capita REAL)''')
    conn.commit()
    conn.close()

def view_database():
    conn = sqlite3.connect('gdp_by_country.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM gdp_by_country_data")
    rows = cursor.fetchall()
    print("Database contents:")
    for row in rows:
        print(row)
    conn.close()

def scrape_gdp_by_country():
    url = "https://www.worldometers.info/gdp/gdp-by-country/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr')
    gdp_by_country_data = []
    for row in rows:
        cells = row.find_all('td')
        if len(cells) >= 7:
            country = cells[1].text.strip()
            gdp = cells[2].text.strip().replace('$', '').replace(',', '')
            population = cells[5].text.strip().replace(',', '')
            gdp_per_capita = cells[6].text.strip().replace('$', '').replace(',', '')
            gdp_by_country_data.append((country, gdp, population, gdp_per_capita))
    return gdp_by_country_data

def read_country_nationality_mapping(csv_file):
    country_nationality_map = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            country_name = row['en_short_name']
            nationality = row['nationality']
            country_nationality_map[country_name] = nationality
    return country_nationality_map

def update_database(csv_file):
    country_nationality_map = read_country_nationality_mapping(csv_file)
    conn = sqlite3.connect('gdp_by_country.db')
    cursor = conn.cursor()
    for country_name, nationality in country_nationality_map.items():
        try:
            cursor.execute("UPDATE gdp_by_country_data SET country = ? WHERE country = ?", (nationality, country_name))
        except sqlite3.IntegrityError:
            print(f"Duplicate entry found for {country_name}, skipping update.")
    conn.commit()
    conn.close()

def main():
    create_database()
    gdp_by_country_data = scrape_gdp_by_country()
    if gdp_by_country_data:
        conn = sqlite3.connect('gdp_by_country.db')
        cursor = conn.cursor()
        try:
            cursor.executemany("INSERT INTO gdp_by_country_data (country, gdp, population, gdp_per_capita) VALUES (?, ?, ?, ?)", gdp_by_country_data)
            conn.commit()
        except sqlite3.IntegrityError:
            print("Duplicate data detected.")
        conn.close()
    update_database('Data/countries.csv')
    view_database()

if __name__ == "__main__":
    main()



Duplicate data detected.


OperationalError: database is locked